# Imports

In [1]:
import json,os,shutil,yaml,torch
import torch.nn as nn
from models import create_model
from utils.util import get_optimizer, get_loss, get_scheduler
from utils.data_container import get_data_loader
from utils.preprocess import preprocessing_for_metric
from utils.train import train_model, test_model2

In [2]:
def train(conf, data_category):
    print(json.dumps(conf, indent=4))

    os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
    device = torch.device(0)

    model_name = conf['model']['name']
    optimizer_name = conf['optimizer']['name']
    data_set = conf['data']['dataset']
    scheduler_name = conf['scheduler']['name']
    loss = get_loss(**conf['loss'])
#     print("aaaa", device)

    loss.to(device)

    support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                           Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
    model, trainer = create_model(model_name,
                                  loss,
                                  conf['model'][model_name],
                                  data_category,
                                  device,
                                  support)

    optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
    scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
    if torch.cuda.device_count() > 1:
        print("use ", torch.cuda.device_count(), "GPUS")
        model = nn.DataParallel(model)
    else:
        model.to(device)

    save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
    run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

    shutil.rmtree(save_folder, ignore_errors=True)
    os.makedirs(save_folder)
    shutil.rmtree(run_folder, ignore_errors=True)
    os.makedirs(run_folder)

    with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
        yaml.safe_dump(conf, _f)

    data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                          model_name=model_name)


    train_model(model=model,
                       dataloaders=data_loader,
                       trainer=trainer,
                       optimizer=optimizer,
                       normal=normal,
                       scheduler=scheduler,
                       folder=save_folder,
                       tensorboard_folder=run_folder,
                       device=device,
                       **conf['train'])
    test_model2(folder=save_folder,
                      trainer=trainer,
                      model=model,
                      normal=normal,
                      dataloaders=data_loader,
                      device=device)


In [3]:
# print(conf['taxi'])

In [4]:
con = 'evoconv2-config'
data = ['taxi']
with open(os.path.join('config', f'{con}.yaml')) as f:
    conf = yaml.safe_load(f)
# train(conf, data)

In [5]:
# conf
data_category=['taxi']

os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
device = torch.device(0)

model_name = conf['model']['name']
optimizer_name = conf['optimizer']['name']
data_set = conf['data']['dataset']
scheduler_name = conf['scheduler']['name']
loss = get_loss(**conf['loss'])

loss.to(device)


support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                       Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
model, trainer = create_model(model_name,
                              loss,
                              conf['model'][model_name],
                              data_category,
                              device,
                              support)

optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
if torch.cuda.device_count() > 1:
    print("use ", torch.cuda.device_count(), "GPUS")
    model = nn.DataParallel(model)
else:
    model.to(device)

save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

shutil.rmtree(save_folder, ignore_errors=True)
os.makedirs(save_folder)
shutil.rmtree(run_folder, ignore_errors=True)
os.makedirs(run_folder)

with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
    yaml.safe_dump(conf, _f)

data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                      model_name=model_name)






std: 37.78643496676519 mean: 26.451628298080365
nodevec1 : torch.Size([266, 50])
nodevec2 : torch.Size([50, 266])
w1 : torch.Size([50, 50])
w2 : torch.Size([50, 50])
b1 : torch.Size([50])
b2 : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.0.out.weight : torch.Size([50, 108])
encoder.0.ru_gate_g_conv.graphconv.0.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.1.out.weight : torch.Size([50, 200])
encoder.0.ru_gate_g_conv.graphconv.1.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.graphconv.2.out.weight : torch.Size([50, 200])
encoder.0.ru_gate_g_conv.graphconv.2.out.bias : torch.Size([50])
encoder.0.ru_gate_g_conv.attlinear.weight : torch.Size([1, 13300])
encoder.0.ru_gate_g_conv.attlinear.bias : torch.Size([1])
encoder.0.candidate_g_conv.graphconv.0.out.weight : torch.Size([25, 108])
encoder.0.candidate_g_conv.graphconv.0.out.bias : torch.Size([25])
encoder.0.candidate_g_conv.graphconv.1.out.weight : torch.Size([25, 100])
encoder.0.candidate_g_conv.graphconv.1.

In [6]:
# print(data_loader)

In [7]:
# print(data_loader)

In [8]:
train_model(model=model,
                   dataloaders=data_loader,
                   trainer=trainer,
                   optimizer=optimizer,
                   normal=normal,
                   scheduler=scheduler,
                   folder=save_folder,
                   tensorboard_folder=run_folder,
                   device=device,
                   **conf['train'])

path does not exist
EvoNN2(
  (encoder): DCRNNEncoder(
    (0): DCGRUCell(
      (ru_gate_g_conv): EvolutionCell(
        (perceptron): ModuleList()
        (graphconv): ModuleList(
          (0): GraphConv_(
            (out): Linear(in_features=108, out_features=50, bias=True)
          )
          (1): GraphConv_(
            (out): Linear(in_features=200, out_features=50, bias=True)
          )
          (2): GraphConv_(
            (out): Linear(in_features=200, out_features=50, bias=True)
          )
        )
        (attlinear): Linear(in_features=13300, out_features=1, bias=True)
      )
      (candidate_g_conv): EvolutionCell(
        (perceptron): ModuleList()
        (graphconv): ModuleList(
          (0): GraphConv_(
            (out): Linear(in_features=108, out_features=25, bias=True)
          )
          (1): GraphConv_(
            (out): Linear(in_features=100, out_features=25, bias=True)
          )
          (2): GraphConv_(
            (out): Linear(in_features=10

train epoch:   0, train loss: 32.1036: : 94it [00:25,  3.66it/s]
validate epoch:   0, validate loss: 41.6438: : 21it [00:01, 17.55it/s]


Better model at epoch 0 recorded.


test  epoch:   0, test  loss: 41.5838: : 21it [00:01, 16.78it/s]
train epoch:   1, train loss: 25.2841: : 94it [00:25,  3.71it/s]
validate epoch:   1, validate loss: 32.249: : 21it [00:01, 15.06it/s] 


Better model at epoch 1 recorded.


test  epoch:   1, test  loss: 31.6402: : 21it [00:01, 15.79it/s]
train epoch:   2, train loss: 18.2249: : 94it [00:25,  3.70it/s]
validate epoch:   2, validate loss: 25.5824: : 21it [00:01, 15.84it/s]


Better model at epoch 2 recorded.


test  epoch:   2, test  loss: 25.2444: : 21it [00:01, 15.84it/s]


In [11]:
import time


In [12]:
start_time = time.time()
test_model2(folder=save_folder,
                  trainer=trainer,
                  model=model,
                  normal=normal,
                  dataloaders=data_loader,
                  device=device)
print("--- %s seconds ---" % (time.time() - start_time))


21it [00:01, 18.78it/s]


test results:
{
    "MAE": {
        "horizon-0": 9.632831283164034,
        "horizon-1": 9.789632721637865,
        "horizon-2": 10.995114162625248,
        "horizon-3": 12.235286964669976,
        "horizon-4": 13.377276435658844,
        "horizon-5": 14.445037475040397,
        "horizon-6": 15.418040851104077,
        "horizon-7": 16.306145775239127,
        "horizon-8": 17.06977013195491,
        "horizon-9": 17.700747273507872,
        "horizon-10": 18.242606378314903,
        "horizon-11": 18.808296508983634
    },
    "RMSE": {
        "horizon-0": 20.017762174973317,
        "horizon-1": 19.380202841464886,
        "horizon-2": 20.521134882471515,
        "horizon-3": 21.937390609844062,
        "horizon-4": 23.400871453015505,
        "horizon-5": 24.8240370742869,
        "horizon-6": 26.147470944666885,
        "horizon-7": 27.291256582664985,
        "horizon-8": 28.184887394896784,
        "horizon-9": 28.847723696862438,
        "horizon-10": 29.41562778636512,
        "ho